In [1]:
from pathlib import Path
import json

import numpy as np
import pandas as pd

In [2]:
data_path = 'spotify_million_playlist_dataset/data'
THREE_K = '_3k'
TWENTY_K = '_20k'
HUNDRED_K = '_100k'

In [3]:
def load_jsons(data_path):
    pathlist = Path(data_path).glob('**/*.json')
    df_slices = []

    for path in pathlist:
        data = json.load(open(path))
        df_slices.append(pd.DataFrame(data['playlists']))

    df = pd.concat(df_slices)
    return df

In [4]:
def playlist_level(df):
    df = df.drop(columns=['tracks'])
    df = df.sort_values(by=['pid'])
    df = df.reset_index().drop(columns=['index'])
    return df

In [5]:
def track_level(df):
    df = df.drop(columns=['name','collaborative', 'modified_at', 'num_tracks', 'num_albums',
                          'num_followers', 'num_edits', 'duration_ms', 'num_artists', 'description'])
    df['tracks'] = df['tracks'].apply(lambda x: pd.DataFrame(x))
    
    tracks_list = []
    playlist_ids = []
    
    for _, row in df.iterrows():
        sub_df = row['tracks']
        tracks_num = len(sub_df)
        tracks_list.append(sub_df)
        playlist_ids.append(np.repeat(row['pid'], tracks_num))
    
    df = pd.concat(tracks_list)
    df['pid'] = np.concatenate(playlist_ids).flatten()
    df = df.sort_values(by=['pid', 'pos'])
    df = df.reset_index().drop(columns=['index'])
    return df

In [6]:
size = HUNDRED_K

In [ ]:
df = load_jsons(data_path)
playlist_df = playlist_level(df)
#track_df = track_level(df)

In [ ]:
# TODO dont save index column
playlist_df.to_csv('csv_data/playlist_df.csv')
#track_df.to_csv('csv_data/track_df' + size + '.csv')